In [96]:
import glob,os,subprocess,json,re
import PIL.Image
from IPython.display import display

def print_js(js):
    print(json.dumps(js,indent=2))

### 先切换文件夹

In [5]:
%cd "./TV Album (2009)(+CDs)_BDrip_x265_flac_dun jpn add chn,yue,eng"

E:\bangumi\K-ON\TV Album (2009)(+CDs)_BDrip_x265_flac_dun jpn add chn,yue,eng


In [3]:
%pwd

'E:\\bangumi\\K-ON'

In [6]:
import os
os.listdir()

['BD Scans',
 'BD Scans.zip',
 'Bonus',
 'Bonus.zip',
 'CDs',
 'CDs.zip',
 'K-ON! 2009 - EP01 [BD 1920x1080 23.976fps AVC-yuv420p10 FLACx3 Chap] - mawen1250.chs.ass',
 'K-ON! 2009 - EP01 [BD 1920x1080 23.976fps AVC-yuv420p10 FLACx3 Chap] - mawen1250.mka',
 'K-ON! 2009 - EP01 [BD 1920x1080 23.976fps AVC-yuv420p10 FLACx3 Chap] - mawen1250.mkv',
 'K-ON! 2009 - EP02 [BD 1920x1080 23.976fps AVC-yuv420p10 FLACx3 Chap] - mawen1250.chs.ass',
 'K-ON! 2009 - EP02 [BD 1920x1080 23.976fps AVC-yuv420p10 FLACx3 Chap] - mawen1250.mka',
 'K-ON! 2009 - EP02 [BD 1920x1080 23.976fps AVC-yuv420p10 FLACx3 Chap] - mawen1250.mkv',
 'K-ON! 2009 - EP03 [BD 1920x1080 23.976fps AVC-yuv420p10 FLACx3 Chap] - mawen1250.chs.ass',
 'K-ON! 2009 - EP03 [BD 1920x1080 23.976fps AVC-yuv420p10 FLACx3 Chap] - mawen1250.mka',
 'K-ON! 2009 - EP03 [BD 1920x1080 23.976fps AVC-yuv420p10 FLACx3 Chap] - mawen1250.mkv',
 'K-ON! 2009 - EP04 [BD 1920x1080 23.976fps AVC-yuv420p10 FLACx3 Chap] - mawen1250.chs.ass',
 'K-ON! 2009 - EP04 

### 以下是重命名某个后缀

In [13]:
# os.listdir()

paths=glob.glob("**/*.jp2",recursive=True)
# paths[-5:]

os.path.exists(paths[0])

def new_path(path):
    tmp=path.split(".")
    tmp[-1]="png"
    return ".".join(tmp)
# print(paths[0],new_path(paths[0]))

[EAC][090422] TVアニメ けいおん! EDテーマ「Don't say“lazy”」／桜高軽音部\Booklet\PCCG-70038_001.jp2 [EAC][090422] TVアニメ けいおん! EDテーマ「Don't say“lazy”」／桜高軽音部\Booklet\PCCG-70038_001.png


In [16]:
img=PIL.Image.open(paths[0])
# display(img)
img.save(new_path(paths[0]))

In [17]:
%%time
for i in paths:
    img=PIL.Image.open(i)
    img.save(new_path(i))

Wall time: 1min 13s


### 下面是整理视频音轨和字幕轨，原则是中文字幕默认和原配优先

#### 方案一，有无字幕视频和下载的不同名字幕

这时一般只有集数有用

注意先测试轴对不对

In [141]:
ass=glob.glob("*.ass",recursive=True)
video=glob.glob("*.mkv",recursive=False)


for i in ass:
    print(i)
for i in video:
    print(i)

Toradora! EP01 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP02 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP03 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP04 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP05 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP06 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP07 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP08 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP09 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP10 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP11 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP12 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1

In [143]:
temp="Toradora! EP{:0>2d} [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.{}"
ass=temp.format(1,"ass")
mkv=temp.format(1,"mkv")

In [144]:
output=subprocess.run(
    [
        "MediaInfo.exe",
        "--Output=JSON",
        mkv,
#         output_file,
    ], stdout=subprocess.PIPE,stderr=subprocess.PIPE,
)

# print("stderr",output.stderr.decode())
# print("stdout\n"+output.stdout.decode())

js=json.loads(output.stdout)
# print(json.dumps(js,indent=2))
# print(len(js["media"]["track"]))

track=js["media"]["track"]
# print_js(track)

filter_key=["@type","@typeorder","StreamOrder","ID","Language","Default","Forced","Title"]

t_filter=[dict(filter(lambda x : x[0] in  filter_key, t.items())) for t in track] # only print keys in list
print_js(t_filter)

[
  {
    "@type": "General"
  },
  {
    "@type": "Video",
    "StreamOrder": "0",
    "ID": "1",
    "Language": "en",
    "Default": "Yes",
    "Forced": "No"
  },
  {
    "@type": "Audio",
    "StreamOrder": "1",
    "ID": "2",
    "Language": "ja",
    "Default": "Yes",
    "Forced": "No"
  },
  {
    "@type": "Menu"
  }
]


In [52]:
cmd=[
    "ffmpeg.exe",
    "-i",
    "{file_mv}",
    "-i",
    "{file_ass}",
    "-map",
    "0:v",
    "-map",
    "0:a",
    "-map",
    "1:s",
    "-map",
    "0:s",
    "-disposition:s:0",
    "default",
    "-disposition:s:1",
    "0",
    "-c",
    "copy",
    "{file_output}"
]
cmd=" ".join(cmd)
print(cmd.format(file_mv=file_mv,file_ass=file_ass,file_output=output_file))

ffmpeg.exe -i [Eupho&VCB-Studio] Baja no Studio ~Baja no Mita Umi~ [Ma10p_1080p][x265_flac].mkv -i [Eupho&VCB-Studio] Baja no Studio ~Baja no Mita Umi~ [Ma10p_1080p][x265_flac].ass -map 0:v -map 0:a -map 1:s -map 0:s -disposition:s:0 default -disposition:s:1 0 -c copy out.mkv


In [60]:
%%time

output_file="out.mkv"
cmd="""\
ffmpeg.exe -i {file_mv} -i {file_ass} -map 0:v -map 0:a -map 1:s -map 0:s \
-disposition:s:0 default -disposition:s:1 0 -c copy {file_output}\
"""#为了好编辑命令

cmd=cmd.split(" ")
# print(cmd)
cmd[cmd.index("{file_mv}")]=file_mv
cmd[cmd.index("{file_ass}")]=file_ass
cmd[cmd.index("{file_output}")]=output_file
# print(cmd)

ff=subprocess.run(cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE,text=True,input="y")

print("stderr\n"+ff.stderr)
print("stdout\n"+ff.stdout)
# print(cmd)

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

In [149]:
temp="Toradora! EP{:0>2d} [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.{}"
temp_output="Toradora! EP{:0>2d} [BD 1080p 23.976fps AVC-yuv420p10 FLAC].mkv"

cmd_temp="""\
ffmpeg.exe -i {file_mv} -i {file_ass} -map 0:v -map 0:a -map 1:s \
-disposition:s:0 default -c copy -metadata:s:s:0 language=zh {file_output}\
"""#为了好编辑命令


cmd=cmd_temp.split(" ")

cmd[cmd.index("{file_mv}")]=temp.format(1,"mkv")
cmd[cmd.index("{file_ass}")]=temp.format(1,"ass")
cmd[cmd.index("{file_output}")]=temp_output.format(1)

ff=subprocess.run(cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE,text=True,input="y")
print("stderr\n"+ff.stderr)
print("stdout\n"+ff.stdout)

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

In [147]:
%%time
temp="Toradora! EP{:0>2d} [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.{}"
temp_output="Toradora! EP{:0>2d} [BD 1080p 23.976fps AVC-yuv420p10 FLAC].mkv"

cmd_temp="""\
ffmpeg.exe -i {file_mv} -i {file_ass} -map 0:v -map 0:a -map 1:s \
-disposition:s:0 default -c copy {file_output}\
"""#为了好编辑命令

for i in range(1,26):
    cmd=cmd_temp.split(" ")

    cmd[cmd.index("{file_mv}")]=temp.format(i,"mkv")
    cmd[cmd.index("{file_ass}")]=temp.format(i,"ass")
    cmd[cmd.index("{file_output}")]=temp_output.format(i)
    
    ff=subprocess.run(cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE,text=True,input="y")
    print("stderr\n"+ff.stderr)
    print("stdout\n"+ff.stdout)

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

...

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

stderr
ffmpeg version 4.4.1-essentials_build-www.gyan.dev Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libmfx --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-libgsm --enable-libopencore-amrnb -

In [23]:
os.remove(file_mv)
os.remove(file_ass)
os.rename(output_file,file_mv)

## 以下是调整一些字幕的字体

In [5]:
%pwd

'E:\\download\\Toradora!'

In [97]:
import glob
for filename in glob.glob('./*.ass'):
    print(filename)
    
ass=glob.glob('./*.ass')

r=r'.*\[(?P<EP>.*?)\]\[X26'

print( re.search( r, ass[0]).groupdict() )
for i in ass:
    print( re.search( r, i).groupdict() )

.\[YYDM-11FANS][Toradora][BDRIP][01][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][02][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][03][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][04][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][05][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][06][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][07][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][08][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][09][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][10][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][11][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][12][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][13][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][14][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][15][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][16][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP][17][X264_AAC][720P].ass
.\[YYDM-11FANS][Toradora][BDRIP

In [119]:
string="""[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Default,方正准圆_GBK,25,&H00FFFFFF,&HF0000000,&H00404080,&H64404080,0,0,0,0,100,100,0,0,1,2,1,2,30,30,10,1
Style: STAFF,黑体,18,&H00FFFFFF,&HF0000000,&H50000000,&H50000000,-1,0,0,0,100,100,0,0,1,1,1,2,30,30,10,1
Style: title,方正大标宋简体,28,&H00EFEFEF,&HF0000000,&H00000000,&H00000000,-1,0,0,0,100,100,0,0,1,0,2,2,30,30,10,1
Style: OPJP,DFGMaruGothic_U-Md,22,&H000F6999,&HF0000000,&H00FFFFFF,&H641385C1,-1,0,0,0,100,100,0,0,1,2,2,7,10,30,10,1
Style: OPCN,经典中圆简,23,&H00FFFFFF,&HF0000000,&H000F6999,&H641385C1,-1,0,0,0,100,100,0,0,1,1,2,3,30,10,10,1
Style: paper,幼圆,13,&H00000000,&HF0000000,&H00000000,&HF0000000,-1,0,0,0,100,100,0,0,1,0,0,9,30,30,10,1
Style: EDJP,DFHSMincho-W7,24,&H00F97C00,&HF0000000,&H00FFFFFF,&H64BD5F00,-1,0,0,0,100,100,0,0,1,2,1,7,10,30,10,1
Style: EDCN,方正大标宋_GBK,26,&H00FFFFFF,&HF0000000,&H00FF8000,&HF0BD5F00,-1,0,0,0,100,100,0,0,1,2,1,3,30,10,10,1

[Events]"""
# print(string)
a=re.search(patten,string,re.M|re.S)

def small_size(style):
    style=style.split("\n")
    font=r"(Style: .*?,.*?,)(?P<size>.*?)(,.*)"
    for num in range(len(style)):
        i=style[num]
        search_obj=re.search(font ,i)
        if search_obj!=None:
            size=search_obj.groupdict()['size']
            style[num]=search_obj.group(1)+str(math.floor(int(size)*0.8))+search_obj.group(3)
    return "\n".join(style)
print(a.group(1),small_size(a.group(1)))

Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Default,方正准圆_GBK,25,&H00FFFFFF,&HF0000000,&H00404080,&H64404080,0,0,0,0,100,100,0,0,1,2,1,2,30,30,10,1
Style: STAFF,黑体,18,&H00FFFFFF,&HF0000000,&H50000000,&H50000000,-1,0,0,0,100,100,0,0,1,1,1,2,30,30,10,1
Style: title,方正大标宋简体,28,&H00EFEFEF,&HF0000000,&H00000000,&H00000000,-1,0,0,0,100,100,0,0,1,0,2,2,30,30,10,1
Style: OPJP,DFGMaruGothic_U-Md,22,&H000F6999,&HF0000000,&H00FFFFFF,&H641385C1,-1,0,0,0,100,100,0,0,1,2,2,7,10,30,10,1
Style: OPCN,经典中圆简,23,&H00FFFFFF,&HF0000000,&H000F6999,&H641385C1,-1,0,0,0,100,100,0,0,1,1,2,3,30,10,10,1
Style: paper,幼圆,13,&H00000000,&HF0000000,&H00000000,&HF0000000,-1,0,0,0,100,100,0,0,1,0,0,9,30,30,10,1
Style: EDJP,DFHSMincho-W7,24,&H00F97C00,&HF0000000,&H00FFFFFF,&H64BD5F00,-1,0,0,0,100,100,0,0,1,2,1,7,10,30,10,1
St

In [131]:
r=r'.*\[(?P<EP>.*?)\]\[X26'

for i in ass:
    if "SP" in re.search(r, i).groupdict()["EP"]:
#         print(i)
        continue
    f=open(i, encoding='UTf-16')
    text="".join(f.readlines())

    style_patten=r"""Styles\](.*?)\[Events"""
    search_obj=re.search(style_patten,text,re.M|re.S)
    style=search_obj.group(1)

    text_=text.replace(style,small_size(style))
    new_file="Toradora! EP{} [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass".format(re.search(r, i).groupdict()["EP"])
    if "OVA" in re.search(r, i).groupdict()["EP"]:
        new_file="Toradora! OVA [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass"
    f=open(new_file,"w")
    f.write(text_)
# for i in glob.glob("*.mkv"):
#     print(i)

In [140]:
temp="Toradora! EP{:0>2d} [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.{}"
for i in range(1,26):
    ass=temp.format(i,"ass")
    mkv=temp.format(i,"mkv")
    print(ass)
    print(mkv)    
    print(os.path.exists(ass),os.path.exists(mkv))

Toradora! EP01 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP01 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.mkv
True True
Toradora! EP02 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP02 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.mkv
True True
Toradora! EP03 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP03 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.mkv
True True
Toradora! EP04 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP04 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.mkv
True True
Toradora! EP05 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP05 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.mkv
True True
Toradora! EP06 [BD 1080p 23.976fps AVC-yuv420p10 FLAC] - VCB-Studio & mawen1250.ass
Toradora! EP06 [BD 1080p 2